In [9]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn import set_config; set_config(display='diagram')
from sklearn.linear_model import LinearRegression

In [10]:
df=pd.read_csv('../raw_data/cleaned_data_040321.csv',encoding='latin-1')

In [11]:
X=df[[
    'BFF_15_1','BFF_15_2','BFF_15_3','BFF_15_4','BFF_15_5','BFF_15_6','BFF_15_7','BFF_15_8','BFF_15_9','BFF_15_10','BFF_15_11',
    'BFF_15_12','BFF_15_13','BFF_15_14','BFF_15_15','Dem_age','Dem_gender','Dem_edu','Dem_edu_mom','Dem_employment','Dem_Expat',
    'Dem_maritalstatus','Dem_riskgroup','Dem_isolation'
]]
y=df['SLON3_avg']

In [12]:
df['Dem_edu'].replace(
        {
            'Uninformative response':0,
            'None':1,
            'Up to 6 years of school':2, 
            'Up to 9 years of school':3,
            'Up to 12 years of school':4, 
            'Some College, short continuing education or equivalent':5,
            'College degree, bachelor, master': 6,
            'PhD/Doctorate':7
        })

0        6
1        6
2        5
3        4
4        6
        ..
85484    6
85485    6
85486    5
85487    4
85488    5
Name: Dem_edu, Length: 85489, dtype: int64

In [13]:
def edu_func(df):
    df['Dem_edu']=df['Dem_edu'].replace(
        {
            'Uninformative response':0,
            'None':1,
            'Up to 6 years of school':2, 
            'Up to 9 years of school':3,
            'Up to 12 years of school':4, 
            'Some College, short continuing education or equivalent':5,
            'College degree, bachelor, master': 6,
            'PhD/Doctorate':7
        })
    return  df[['Dem_edu']]

def edu_mom_func(df):    
    df['Dem_edu_mom'] = df['Dem_edu_mom'].replace({
        'Uninformative response':0,
        'None':1,
        'Up to 6 years of school':2,
        'Up to 9 years of school':3,
        'Up to 12 years of school':4,
        'Some College or equivalent':5,
        'College degree': 6,
        'PhD/Doctorate':7
    }
    )
    return  df[['Dem_edu_mom']]


def risk_group_func(df): 
    df['Dem_riskgroup'] = df['Dem_riskgroup'].replace({'No':1,'Not sure':2, 'Yes':3})    
    return  df[['Dem_riskgroup']]

def expat_func(df):
    df['Dem_Expat'] = df['Dem_Expat'].replace({'no':0,'yes':1})
    return df[['Dem_Expat']]

In [16]:
preprocessor = ColumnTransformer(
    [    
        ('edu', FunctionTransformer(edu_func), ['Dem_edu']),
        ('edu_mom', FunctionTransformer(edu_mom_func), ['Dem_edu_mom']),
        ('risk_group', FunctionTransformer(risk_group_func), ['Dem_riskgroup']),
        ('expat', FunctionTransformer(expat_func), ['Dem_Expat']),
        ('ohe_transformer', OneHotEncoder(), ['Dem_gender', 'Dem_maritalstatus','Dem_employment','Dem_isolation'])
    ],remainder='passthrough'
)

base_pipe = Pipeline([
    ('imputer', preprocessor),
    ('scaler', MinMaxScaler())
])



loneliness_pipe = Pipeline([
    ('preprocessing', base_pipe),
    ('linear_regression',  LinearRegression())
])

In [17]:
loneliness_pipe.fit(X,y)

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('edu',
                                                                   FunctionTransformer(func=<function edu_func at 0x7f90547b1670>),
                                                                   ['Dem_edu']),
                                                                  ('edu_mom',
                                                                   FunctionTransformer(func=<function edu_mom_func at 0x7f90547b14c0>),
                                                                   ['Dem_edu_mom']),
                                                                  ('risk_group',
                                                                   FunctionTransformer(func=<function risk_group_func at 0x7f90547b1790>),
                                                                   ['Dem_riskgroup']),
                                                                  ('expat',
                                                                   FunctionTransformer(func=<function expat_func at 0x7f90547b18b0>),
                                                                   ['Dem_Expat']),
                                                                  ('ohe_transformer',
                                                                   OneHotEncoder(),
                                                                   ['Dem_gender',
                                                                    'Dem_maritalstatus',
                                                                    'Dem_employment',
                                                                    'Dem_isolation'])])),
                                 ('scaler', MinMaxScaler())])),
                ('linear_regression', LinearRegression())])

In [18]:
import joblib 
joblib.dump(loneliness_pipe, 'model_linear_loneliness_3.joblib')

['model_linear_loneliness_3.joblib']

In [20]:
import pickle
# Export pipeline as pickle file
with open("loneliness_pipeline.pkl", "wb") as file:
    pickle.dump(loneliness_pipe, file)